In [42]:
# Se importan las librerías necesarias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

pd.options.mode.chained_assignment = None

In [64]:
# Se leen los dataframes.
postulantesEducacion = pd.read_csv('../Data/fiuba_1_postulantes_educacion.csv')
postulantesGeneroYEdad = pd.read_csv('../Data/fiuba_2_postulantes_genero_y_edad.csv')
oportunidadVistas = pd.read_csv('../Data/fiuba_3_vistas.csv')
oportunidadPostulaciones = pd.read_csv('../Data/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('../Data/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('../Data/fiuba_6_avisos_detalle.csv')

In [69]:
# Elimino los postulantes cuyos ids o fechas de nacimiento son nulas.

# Elimino las filas que no tengan id del postulante.
postulantesEducacion.dropna(subset = ['idpostulante'], inplace = True)
postulantesGeneroYEdad.dropna(subset = ['idpostulante'], inplace = True)

# Creo un nuevo dataframe con los postulantes que tengan fecha de nacimiento.
postulantesConEdad = postulantesGeneroYEdad.dropna(subset=['fechanacimiento'])

In [70]:
# Creo una nueva columna con la edad del postulante. Descarto los postulantes cuyas edades no pueden ser averiguadas.

# Paso la columna fecha de nacimiento a un datetime.
postulantesConEdad['fechanacimiento'] = pd.to_datetime(postulantesConEdad['fechanacimiento'], errors='coerce')

# Creo la columna edad como diferencia entre la fecha actual y la fecha de nacimiento.
fechaActual = pd.Timestamp(datetime.datetime.now())
postulantesConEdad['edad'] = (hoy - postulantesConEdad['fechanacimiento']).astype('<m8[Y]')

# Elimino los postulantes sin edades.
postulantesConEdad.dropna(subset = ['edad'], inplace = True)

# Descarto la columna fecha de nacimiento.
postulantesConEdad.drop('fechanacimiento', 1, inplace = True)

In [77]:
# Creo un unico dataframe para los datos del postulante.

# Renombro las columnas del dataframe de educacion a algo mas descriptivo.
postulantesEducacion.rename(columns={'nombre': 'niveleducativo', 'estado': 'estadoniveleducation'}, inplace = True)

# Hago un left join tomando todos los registros del dataframe que contiene la edad.
postulantesDatos = pd.merge(postulantesConEdad, postulantesEducacion, on='idpostulante', how='left')

In [80]:
avisosDetalle.sample(15)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
8222,1111729955,1,Vendedor de local de indumentaria infantil (Pa...,<p>Nos encontramos en la búsqueda de vendedore...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ventas,Mimo & Co.
2189,1112346942,1,OPERARIO METALURGICO,<p>ESTAMOS EN LA BUSQUEDA DE UN OPERARIO METAL...,Gran Buenos Aires,NaN,NaN,Full-time,Otro,Producción,INDUSTRIAS CARAPACHAY S.A.
2310,1112348918,1,Analista SAP MM/PP,<p>Cafés La Virginia S.A se encuentra en la bú...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Análisis Funcional,La Virginia
4229,1112296027,1,Analista de Testing manual Ssr. // Zona Recole...,<p>En <strong>SOLUTIX</strong> seleccionamos <...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Testing / QA / QC,SOLUTIX S.A.
6002,1112238499,1,Sistemas/Administración (Sistema TANGO),<p>Requisitos:<br />(Por favor NO aplicar para...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Administración,Sepelios Alvear
752,1112217039,1,Analista Programador PHP - Zona Sur,<p>Para Sanatorio privado dedicado a brindar s...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Aliantec
5353,1112365641,1,Técnico Electrónico,"<p><strong><span style="""">EKKON</span></strong...",Gran Buenos Aires,NaN,NaN,Full-time,Junior,Oficios y Profesiones,Ekkon Expertos
138,1111689385,1,Analista de Siniestros - Importante Cía de Seg...,"<p><strong>Importante Compañía de Seguros, bu...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Administracion de Seguros,Avatar-RH
3789,1112234400,1,Operarios y Mecánicos p/ Empresa Textil Z/ Bur...,<p>Importante Empresa Textil se encuentra en l...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Producción,Suministra
9811,1112320236,1,Cadete Administrativo,Depósito de cheques en entidades bancarias.Dep...,Capital Federal,NaN,Venezuela 676,Full-time,Otro,Cadetería,Geosistemas S.R.L.


In [81]:
oportunidadVistas.sample(15)

,idAviso,timestamp,idpostulante
867631,1112341841,2018-02-28T13:52:14.844-0500,dY95zeX
245073,1111959242,2018-02-25T21:02:09.083-0500,ekNqkKj
763033,1111753347,2018-02-28T08:16:08.219-0500,owEVk34
888394,1112368339,2018-02-28T16:20:15.374-0500,e3DZZr
784478,1112299205,2018-02-28T08:13:52.677-0500,xkOrr8b
719386,1112364688,2018-02-27T20:39:50.800-0500,1av00z
391756,1112295684,2018-02-26T13:47:57.215-0500,akjmG8q
109701,1112344695,2018-02-24T11:25:41.369-0500,bO4lj8E
39928,1112354079,2018-02-23T18:16:30.283-0500,GNZvYE9
915751,1112345347,2018-02-28T18:37:40.732-0500,LN8vWAr


In [82]:
oportunidadPostulaciones.sample(15)

,idaviso,idpostulante,fechapostulacion
483510,1112226419,GNX25N8,2018-01-23 12:51:00
3110563,1112323404,8MPZpEW,2018-02-13 22:58:13
2919526,1112345157,OqmbMY8,2018-02-19 11:33:31
1129030,1112362764,8Zr1GD,2018-02-26 14:16:22
1918010,1112214364,8M2Yj8D,2018-01-18 08:42:23
438779,1112351961,ZBPO0b,2018-02-22 18:37:13
2877213,1112233775,bOjYVvY,2018-01-18 20:34:41
3165165,1112365691,aVZNdJ,2018-02-28 20:56:40
412966,1112204682,vVj1dXX,2018-01-31 17:21:51
1205736,1112291379,2zQZ98Q,2018-02-23 05:40:26
